In [ ]:
from datetime import datetime

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from db_utils import get_rect_to_rect_data
from cluster_utils import cluster_by_max_projection_and_distance

In [ ]:
ped_files = ['VM2_-2112701535', 'VM2_-217686115','VM2_-1247665811','VM2_-104300786', 'VM2_330973206', 'VM2_1100569031', 'VM2_421371629', 'VM2_421371629', 'VM2_1476499235', 'VM2_-1451152685', 'VM2_-1523872256']

In [ ]:
# from alexanderstr onto alexanderstr, south to west --> 255 rides
start_rect_coords = (13.416169,52.521508,13.416513,52.521832)
end_rect_coords = (13.41591,52.522275,13.416421,52.522569)

# from karl-marx onto alexanderstr, east to south --> 20 rides
# start_rect_coords = (13.41673,52.5219,13.417469,52.522088)
# end_rect_coords = (13.416021,52.521813,13.416321,52.52203)
# end_rect_coords = (13.415876,52.521774,13.416321,52.52203) # larger endbox

end_date_str = '2020-01-01 00:00:00'
end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M:%S')

# df_simra = get_rect_to_rect_data(start_rect_coords, end_rect_coords, end_date=end_date, files_to_exclude=ped_files)
# df_simra = get_rect_to_rect_data(start_rect_coords, end_rect_coords, end_date=end_date, files_to_exclude=None)
df_simra = get_rect_to_rect_data(start_rect_coords, end_rect_coords, files_to_exclude=ped_files)

In [ ]:
# Some entries contain nans, or no speed, even though a distance is given. Inspect further. Option for filtering or preprocessing.

# df_simra.loc[(df_simra==0).any(axis=1)]
# df_simra.loc[(df_simra.isna()).any(axis=1)]

# df_simra[(df_simra.velo == 0) | (df_simra.velo.isna())]
df_simra[((df_simra.velo == 0) | (df_simra.velo.isna())) & (df_simra.dist != 0.0)]

In [ ]:
cluster_by_max_projection_and_distance(df_simra)